### For this file, our goal is to clean the data and return a dataframe for the dataset

In [83]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from dataclasses import dataclass
import os

from dataclasses import dataclass

@dataclass
class Patient:
    RecordID: int = 0
    Age: float = -1
    Gender: int = -1
    ICUType: int = -1
    Height: int = -1
    Albumin: float = -1
    ALP: float = -1
    ALT: float = -1
    AST: float = -1
    Bilirubin: float = -1
    BUN: float = -1
    Cholesterol: float = -1
    Creatinine: float = -1
    DiasABP: float = -1
    FiO2: float = -1
    GCS: int = -1
    Glucose: float = -1
    HCO3: float = -1
    HCT: float = -1
    HR: int = -1
    K: float = -1
    Lactate: float = -1
    Mg: float = -1
    MAP: float = -1
    MechVent: bool = False
    Na: float = -1
    NIDiasABP: float = -1
    NIMAP: float = -1
    NISysABP: float = -1
    PaCO2: float = -1
    PaO2: float = -1
    pH: float = -1
    Platelets: float = -1
    RespRate: int = -1
    SaO2: float = -1
    SysABP: float = -1
    Temp: float = -1
    TroponinI: float = -1
    TroponinT: float = -1
    Urine: float = -1
    WBC: float = -1
    Weight: int = -1



train_path = '../data/dataset/set-a'
label_path = '../data/dataset/Outcomes-a.txt'

files = [f for f in os.listdir(train_path) if f.endswith(".txt")]
files.sort()

patient_data =[]
patient_outcomes =[]
for i in enumerate(files):
    data_path = os.path.join(train_path, files[i[0]])
    patient_data.append( pd.read_csv(data_path))

# patient_data[0]
print(patient_data[0].shape)
print(patient_data[0].columns) 
patient_data[3999].Value[0]


(273, 3)
Index(['Time', 'Parameter', 'Value'], dtype='object')


np.float64(142673.0)

In [84]:
patient_outcomes = pd.read_csv(label_path)
Mortality_labels = patient_outcomes["In-hospital_death"]
Mortality_labels

0       0
1       0
2       0
3       0
4       0
       ..
3995    0
3996    0
3997    0
3998    1
3999    0
Name: In-hospital_death, Length: 4000, dtype: int64

In [85]:
not_to_be_added = [
    'Age',              # Fixed, does not change
    'Gender',           # Binary, constant
    'ICUType',          # Constant for the patient's ICU stay
    'Height',           # Fixed, does not change
    'RecordID',         # Unique identifier, does not change
    'MechVent',         # Binary, should not be summed
    'GCS',              # Score, should not be summed
    'Temp',             # Single measurement at each time point
    'PaCO2',            # Arterial gas, not cumulative
    'PaO2',             # Arterial gas, not cumulative
    'pH',               # Arterial gas, not cumulative
    'Platelets',        # Blood count, should not be summed
    'Weight'            # Fixed for most cases
]
Patient_info = [Patient() for _ in range(len(patient_data))]
for index, patient in enumerate(Patient_info):
     print(int(patient_data[index].Value[0]))
     for _, row in patient_data[index].iterrows():
        param = row['Parameter']
        value = row['Value'] 
        if hasattr(patient, param):
            current = getattr(patient, param)
            if current == -1:
                current = 0
            if param not in not_to_be_added:
                setattr(patient, param, current + value)
            else:
                setattr(patient, param, value)
        else:
            print(f"Warning: Patient {patient.RecordID} has no attribute '{param}'")
print("====\n",Patient_info[0].Weight)

132539
132540
132541
132543
132545
132547
132548
132551
132554
132555
132556
132567
132568
132570
132573
132575
132577
132582
132584
132585
132588
132590
132591
132592
132595
132597
132598
132599
132601
132602
132605
132610
132612
132614
132615
132617
132618
132622
132623
132632
132634
132635
132636
132637
132639
132642
132644
132647
132648
132650
132653
132658
132659
132662
132663
132664
132666
132669
132671
132680
132682
132683
132685
132686
132688
132694
132695
132698
132700
132703
132704
132705
132708
132710
132717
132720
132721
132722
132726
132729
132732
132737
132738
132744
132745
132747
132758
132765
132766
132767
132769
132772
132773
132774
132776
132777
132780
132781
132783
132786
132790
132791
132792
132797
132798
132799
132801
132805
132806
132810
132811
132812
132813
132814
132819
132821
132826
132827
132830
132832
132835
132836
132837
132838
132842
132844
132845
132848
132850
132851
132852
132853
132855
132857
132859
132860
132863
132864
132865
132866
132872
132877
132878

In [86]:
for patient in Patient_info:
    for field in patient.__dataclass_fields__:
        value = getattr(patient, field)
        if value == -1:
            setattr(patient, field, None)
# we make the value None to mask the data, -1 would cause very severe inaccuracies
df = pd.DataFrame([patient.__dict__ for patient in Patient_info])
df

,RecordID,Age,Gender,ICUType,Height,Albumin,ALP,ALT,AST,Bilirubin,...,Platelets,RespRate,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight
0,132539.0,54.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,185.0,732.0,NaN,NaN,37.8,NaN,NaN,6500.0,20.6,NaN
1,132540.0,76.0,1.0,2.0,175.3,NaN,NaN,NaN,NaN,NaN,...,135.0,NaN,581.0,7712.0,37.1,NaN,NaN,6214.0,33.8,81.6
2,132541.0,44.0,0.0,3.0,NaN,5.0,232.0,166.0,399.0,5.8,...,113.0,NaN,95.0,2011.0,37.2,NaN,NaN,5123.0,14.1,56.7
3,132543.0,68.0,1.0,3.0,180.3,4.4,105.0,12.0,15.0,0.2,...,284.0,912.0,NaN,NaN,37.0,NaN,NaN,3275.0,28.2,84.6
4,132545.0,88.0,0.0,3.0,NaN,3.3,NaN,NaN,NaN,NaN,...,97.0,920.0,NaN,NaN,36.7,NaN,NaN,2361.0,8.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,142665.0,70.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,218.0,NaN,NaN,6096.0,36.9,NaN,NaN,1980.0,29.0,87.0
3996,142667.0,25.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,118.0,388.0,NaN,NaN,36.8,NaN,NaN,4675.0,14.2,166.4
3997,142670.0,44.0,1.0,3.0,NaN,NaN,51.0,20.0,20.0,0.5,...,229.0,NaN,NaN,754.0,38.4,NaN,NaN,4193.0,33.2,109.0
3998,142671.0,37.0,1.0,3.0,NaN,3.1,507.0,5913.0,5056.0,2.2,...,314.0,NaN,NaN,8664.0,37.4,NaN,NaN,146.0,52.1,87.4


In [87]:
# from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(strategy='mean')  # You can try 'median' or 'most_frequent' too
# X = df.drop(columns=['RecordID'])  # drop ID from features
# X_imputed = imputer.fit_transform(X)

In [88]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import svm
# from sklearn import linear_model
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
# import matplotlib.pyplot as plt
# X_train, X_test, y_train, y_test = train_test_split(X_imputed, Mortality_labels, test_size=0.2, random_state=42)

# model = DecisionTreeClassifier()
# model.fit(X_train,y_train)
# predictions = model.predict(X_test)
# cm = confusion_matrix(y_test, predictions)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Survived", "Died"])
# disp.plot(cmap='Blues')
# plt.title("Confusion Matrix")
# plt.show()

In [96]:
def raw_to_df(patient_data):
    """
    Convert patient data to a pandas DataFrame.

    Parameters:
        patient_data (Any): Raw patient data after being read as a df from read_csv function

    Returns:
        pd.DataFrame: Structured patient data as a DataFrame.
    """
    not_to_be_added = [
    'Age',              # Fixed, does not change
    'Gender',           # Binary, constant
    'ICUType',          # Constant for the patient's ICU stay
    'Height',           # Fixed, does not change
    'RecordID',         # Unique identifier, does not change
    'MechVent',         # Binary, should not be summed
    'GCS',              # Score, should not be summed
    'Temp',             # Single measurement at each time point
    'PaCO2',            # Arterial gas, not cumulative
    'PaO2',             # Arterial gas, not cumulative
    'pH',               # Arterial gas, not cumulative
    'Platelets',        # Blood count, should not be summed
    'Weight'            # Fixed for most cases
    ]
    Patient_info = [Patient() for _ in range(len(patient_data))]
    for index, patient in enumerate(Patient_info):
        for _, row in patient_data[index].iterrows():
            param = row['Parameter']
            value = row['Value'] 
            if hasattr(patient, param):
                current = getattr(patient, param)
                if current == -1:
                    current = 0
                if param not in not_to_be_added:
                    setattr(patient, param, current + value)
                else:
                    setattr(patient, param, value)
            else:
                print(f"Warning: Patient {patient.RecordID} has no attribute '{param}'")
    for patient in Patient_info:
        for field in patient.__dataclass_fields__:
            value = getattr(patient, field)
            if value == -1:
                setattr(patient, field, None)
    # we make the value None to mask the data, -1 would cause very severe inaccuracies
    df = pd.DataFrame([patient.__dict__ for patient in Patient_info])
    return df

In [ ]:
df2 = raw_to_df(patient_data)
df2

,RecordID,Age,Gender,ICUType,Height,Albumin,ALP,ALT,AST,Bilirubin,...,Platelets,RespRate,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight
0,132539.0,54.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,185.0,732.0,NaN,NaN,37.8,NaN,NaN,6500.0,20.6,NaN
1,132540.0,76.0,1.0,2.0,175.3,NaN,NaN,NaN,NaN,NaN,...,135.0,NaN,581.0,7712.0,37.1,NaN,NaN,6214.0,33.8,81.6
2,132541.0,44.0,0.0,3.0,NaN,5.0,232.0,166.0,399.0,5.8,...,113.0,NaN,95.0,2011.0,37.2,NaN,NaN,5123.0,14.1,56.7
3,132543.0,68.0,1.0,3.0,180.3,4.4,105.0,12.0,15.0,0.2,...,284.0,912.0,NaN,NaN,37.0,NaN,NaN,3275.0,28.2,84.6
4,132545.0,88.0,0.0,3.0,NaN,3.3,NaN,NaN,NaN,NaN,...,97.0,920.0,NaN,NaN,36.7,NaN,NaN,2361.0,8.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,142665.0,70.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,218.0,NaN,NaN,6096.0,36.9,NaN,NaN,1980.0,29.0,87.0
3996,142667.0,25.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,118.0,388.0,NaN,NaN,36.8,NaN,NaN,4675.0,14.2,166.4
3997,142670.0,44.0,1.0,3.0,NaN,NaN,51.0,20.0,20.0,0.5,...,229.0,NaN,NaN,754.0,38.4,NaN,NaN,4193.0,33.2,109.0
3998,142671.0,37.0,1.0,3.0,NaN,3.1,507.0,5913.0,5056.0,2.2,...,314.0,NaN,NaN,8664.0,37.4,NaN,NaN,146.0,52.1,87.4


#### So the main takeaway from this file is the *********raw_to_df********* function
### All done for now 🥳🎉